In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('rocket.csv')
df.head() # Merge 한 원본데이터 가져오기

In [ ]:
df = df.drop(['school','project','language','award','certificate'],axis=1) # 다른 전처리요소는 제거하기

In [ ]:
df = df.dropna()
df= df.reset_index(drop=True) # 경력이 없는 사람은 제거하고, Index 재정렬

In [ ]:
data = pd.DataFrame(columns=['career'])
for k in range(len(df['career'])):
    cr = list(map(lambda x : x.replace(' ',''), df['career'][k].split('\n')))  # \n을 기준으로 나누자 !
    crd= []
    for i in range(2,len(cr)):
        if len(cr[i]) > 0:
            crd.append(cr[i])
    cd = pd.DataFrame(crd,columns=['career'])
    if  0 not in cd[cd['career'].str.contains('년','월',na=False)].index: # 회사랑 시간뽑아내자 ! 
        cdd = cd.loc[cd[cd['career'].str.contains('년','월',na=False)].index-1] # 회사인덱스
        cddd = cd.loc[cd[cd['career'].str.contains('년','월',na=False)].index] # 시간인덱스
        cddd.columns=['time']
        unna = pd.DataFrame(np.full((len(cd.loc[cd[cd['career'].str.contains('년','월',na=False)].index-1]), 1), df['Unnamed: 0'][k]),columns=['Unnamed: 0'])
        cd = pd.concat([cdd.reset_index(drop=True),unna,cddd.reset_index(drop=True)],axis=1)
        d =pd.DataFrame(sorted(list(cd.index),reverse=True),columns=['turnover']) # 이건 원래 이직횟수 고려하려고 넣은건데, 잘못된 데이터까지 회사로 고려해버려서 나중에 다시 구함 !
        cd = pd.concat([cd,d],axis=1)
        data = data.append(cd)
    else :
        print(k)
    
data.columns = ['career','ID','time','turnover']

In [ ]:
data.head()

In [ ]:
# 데이터에서 특수문자들 제거하기
data =data.reset_index(drop=True)
data['career'] = pd.DataFrame(map(lambda x : x.replace('\r',''),data['career']))
data['time']=pd.DataFrame(map(lambda x : x.replace('\r',''),data['time']))
data['career'] =pd.DataFrame(map(lambda x : x.replace('=-',''),data['career']))

In [ ]:
# time을 2020년6월-현재	4개월 이렇게 언제부터언제까지 근무했는지랑 얼마나 근무했는지로 나누자
data = pd.concat([data,pd.DataFrame(map(lambda x : x.split('|'),data['time']))],axis=1)

In [ ]:
data.head()

In [ ]:
data.columns= ['ID','career','time_2','turnover','time','period']

In [ ]:
# career 에서 len이 지나치게 긴것들을 제거했음 (60을 기준으로) 실제 확인해보니까 4개 빼고 전부다 이상치
idx = []
check =[]
for k in range(len(data['career'])):
    if type(data['career'][k]) == str:
        if len(data['career'][k]) >60:
            idx.append(k)
    else:
        check.append(k)

In [ ]:
# check가 이상치임 눈으로 확인해보라구 이상치 인덱스를 check로 따로 담음
data = data.drop(check)
data

In [ ]:
data.reset_index(drop=True,inplace=True)

In [ ]:
# len 18을 기준으로 time 이상치 제거했음 ( 이것또한 check에 담아서 눈으로 확인할수있게 함 )
idx = []
check= []
for k in range(len(data['time'])):
    if type(data['time'][k]) == str:
        if len(data['time'][k]) >18:
            idx.append(k)
    else:
        print(k)
        check.append(k)

In [ ]:
data = data.drop(idx).reset_index(drop=True)
data

In [ ]:
# period에 개월이 없는게 은근 있다 (재직중) 인듯
data = data.dropna(subset=['period']) # period에 결측치 있는거 삭제(3000개)
data

In [ ]:
import datetime as dt

In [ ]:
# 이제 datetime으로 바꾸기 위해서 '-' 을 기준으로 시작날짜 끝날짜 만들자
aa = pd.DataFrame(map(lambda x : x.split('-'),data['time']))
aa.columns= ['start','end']
aa

In [ ]:
# datetime 형식에 맞게 replace
aa['start'] =pd.DataFrame(map(lambda x: x.replace('년','-'),aa['start']))
aa['start'] =pd.DataFrame(map(lambda x: x.replace('월','-01'),aa['start']))
aa['start']

In [ ]:
# datetime에서  len < 9보다 크면 이상치로 간주했음 ! 근데 없었음 ! 이미 이전에 많이 제거해서 필요없는듯 ~!
idx = []
check= []
for k in range(len(aa['start'])):
    if type(aa['start'][k]) == str:
        if len(aa['start'][k]) <9:
            idx.append(k)
    else:
        print(k)
        check.append(k)

In [ ]:
aa = aa.drop(idx).reset_index(drop=True)
aa

In [ ]:
# datetime으로 바꿨음 ~!
st =pd.DataFrame(map(lambda x : dt.datetime.strptime(x,'%Y-%m-%d'),aa['start']))
st.head()

In [ ]:
# 끝나는 날도 똑같이 ~! 현재는 10월 1일로 했는데 수정가능함
aa['end'] =pd.DataFrame(map(lambda x: x.replace('년','-'),aa['end']))
aa['end'] =pd.DataFrame(map(lambda x: x.replace('월','-01'),aa['end']))
aa['end'] =pd.DataFrame(map(lambda x: x.replace('현재','2020-10-01'),aa['end']))
aa['end']

In [ ]:
ed=pd.DataFrame(map(lambda x : dt.datetime.strptime(x,'%Y-%m-%d'),aa['end']))
ed.head()

In [ ]:
time =pd.concat([st,ed],axis=1)
time.columns = ['start','end']
time

In [ ]:
# 그리고 다시 datetime으로 바꾼걸 데이터프레임에 붙여줍니다
data_3 =pd.concat([data.drop(idx).reset_index(drop=True),time],axis=1)
data_3.head()

In [ ]:
# 이제 필요없게된 time 제거
data_3=data_3.drop(['time_2','time'],axis=1)
data_3.head()

In [ ]:
 # period를 전부다 개월수로 변환하자
bb = data_3[['period','ID']]
bb['period']=pd.DataFrame(map(lambda x: x.replace('-',''),bb['period']))
bb.columns=['period','ID']
bb

In [ ]:
# 왜 개월이랑 개월 아닌걸로 나누냐면 그냥 1년 이렇게 되어있는 데이터를 구분하려고 ! 왜 구분하냐면 아래에서 쓸 eval 함수 때문에 ! 개월수가 있는건 *12+ 하고
# 없으면 *12 만 함 ! 
idx =bb[bb['period'].str.contains('개월',na=False)].index
idx2 = bb[~bb['period'].str.contains('개월',na=False)].index
len(idx)+len(idx2)

In [ ]:
for i in range(len(bb['period'])):
    if i in idx:
        bb['period'][i] =bb['period'][i].replace('년','*12+')
    else :
        bb['period'][i] =bb['period'][i].replace('년','*12')
bb['period']=pd.DataFrame(map(lambda x: x.replace('개월',''),bb['period']))
bb['period']=pd.DataFrame(map(lambda x: eval(x),bb['period']))
bb

In [ ]:
data_4 = pd.concat([data_3.reset_index(drop=True),bb.drop(['ID'],axis=1)],axis=1)
data_4

In [ ]:
# turn(이직횟수) 구하는 과정 ! ID로 인덱싱해서 뽑은다음에 거꾸로 정렬해서 그걸 변수로 뽑기
id =data_4['ID'].unique()
b= pd.DataFrame(columns = ['ID', 'career', 'turnover', 'period', 'start', 'end', 'period', 'turn'])
for i in id:
    a = pd.concat([data_4[data_4['ID'] == i].sort_values(by='start',ascending=True).reset_index(drop=True),pd.DataFrame(data_4[data_4['ID'] == i].sort_values(by='start',ascending=True).reset_index(drop=True).index,columns=['turn'])],axis=1)
    b = b.append(a)
b

In [ ]:
# 필요없게된 period2랑 turnover 제거 !
data_4 = b
data_4.columns = ['ID', 'career', 'turnover', 'period2', 'start', 'end', 'period', 'turn']
data_4 = data_4.drop(['turnover','period2'],axis=1)
data_4 = data_4.reset_index(drop=True)
data_4

In [ ]:
data_4['period'].unique()

In [ ]:
data_4['sum_peri']=0

In [ ]:
# 누적근무기간 구하기
data_4['period'] = pd.to_numeric(data_4['period'])
data_4['sum_peri'] = data_4.groupby('ID')['period'].cumsum()

In [ ]:
data_4

In [ ]:
data_4.to_csv('data_new.csv')